<a href="https://colab.research.google.com/github/vant7e/NIH_MEG/blob/Image_classification/testttt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install tensorflow-hub

In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
import urllib.request
import ssl

# Disable SSL certificate verification for this request
ssl._create_default_https_context = ssl._create_unverified_context

# Download the class label mapping file
label_map_url = "https://download.tensorflow.org/data/ImageNetLabels.txt"
label_map_file = "ImageNetLabels.txt"
urllib.request.urlretrieve(label_map_url, label_map_file)

# Read the class labels from the mapping file
class_labels = []
with open(label_map_file, "r") as file:
    class_labels = [line.strip() for line in file]

# Load the pre-trained MobileNetV2 model
model_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
m = tf.keras.Sequential([
    hub.KerasLayer(model_url)
])
m.build([None, 224, 224, 3])

# Path to the directory containing the uploaded images in Colab
image_dir = '/content/images'

# List all image files in the directory
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Open a text file for saving results
with open('classification_results.txt', 'w') as result_file:
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)

        # Load and preprocess the image
        image = Image.open(image_path)
        image = image.resize((224, 224))
        image = np.array(image) / 255.0
        image = image[tf.newaxis, ...]

        # Perform image classification using the MobileNetV2 model
        result = m.predict(image)

        # Get the predicted class index
        predicted_class_index = np.argmax(result)

        # Get the corresponding label from the mapping file
        predicted_label = class_labels[predicted_class_index]

        # Print the result with the predicted label
        print(f"Classification result for {image_file}: {predicted_label}")

        # Save the result with the predicted label to the text file
        result_file.write(f"Classification result for {image_file}: {predicted_label}\n")



In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
import urllib.request
import ssl

# Disable SSL certificate verification for this request
ssl._create_default_https_context = ssl._create_unverified_context

# Download the class label mapping file
label_map_url = "https://download.tensorflow.org/data/ImageNetLabels.txt"
label_map_file = "ImageNetLabels.txt"
urllib.request.urlretrieve(label_map_url, label_map_file)

# Read the class labels from the mapping file
class_labels = []
with open(label_map_file, "r") as file:
    class_labels = [line.strip() for line in file]

# Load the pre-trained MobileNetV2 model
model_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
m = tf.keras.Sequential([
    hub.KerasLayer(model_url)
])
m.build([None, 224, 224, 3])

# Path to the directory containing the uploaded images in Colab
image_dir = '/content/images'

# List all image files in the directory
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Open a text file for saving results
with open('classification_results.txt', 'w') as result_file:
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)

        # Load and preprocess the image
        image = Image.open(image_path)
        image = image.resize((224, 224))
        image = np.array(image) / 255.0
        image = image[tf.newaxis, ...]

        # Perform image classification using the MobileNetV2 model
        result = m.predict(image)

        # Get the top N predicted class indices and their corresponding labels and confidence scores
        N = 7  # You can change N to get more than 5 results
        top_N_indices = np.argsort(result[0])[::-1][:N]
        top_N_labels = [class_labels[i] for i in top_N_indices]
        top_N_scores = result[0][top_N_indices]

        # Print and save the top N results with labels and confidence scores
        print(f"Classification results for {image_file}:")
        result_file.write(f"Classification results for {image_file}:\n")
        for label, score in zip(top_N_labels, top_N_scores):
            print(f"- Label: {label}, Confidence: {score:.2f}")
            result_file.write(f"- Label: {label}, Confidence: {score:.2f}\n")
        print()
        result_file.write("\n")

In [ ]:
!pip install tensorflow==2.14.0
!pip install tensorflow-hub


In [ ]:
!tf.reset_default_graph()

/bin/bash: -c: line 2: syntax error: unexpected end of file


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image
import urllib.request
import os

# Path to the directory containing the images
image_dir = '/content/images'

# List all image files in the directory
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Define a function to perform object detection on a single image
def perform_object_detection(image_path):
    # Load the object detector model within the function
    detector = hub.load("https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1")

    # Load and preprocess the image
    image = Image.open(image_path)
    image = np.array(image)

    # Prepare the image tensor
    image_tensor = tf.convert_to_tensor(image, dtype=tf.float32)

    # Perform object detection
    detector_output = detector(image_tensor, as_dict=True)

    # Extract the top 5 detections
    top_N = tf.math.top_k(detector_output['detection_scores'][0], k=5)
    detection_boxes = tf.gather(detector_output['detection_boxes'][0], top_N.indices)
    detection_class_entities = tf.gather(detector_output['detection_class_entities'][0], top_N.indices)
    detection_class_names = tf.gather(detector_output['detection_class_names'][0], top_N.indices)
    detection_class_labels = tf.gather(detector_output['detection_class_labels'][0], top_N.indices)
    detection_scores = top_N.values

    return {
        'detection_boxes': detection_boxes,
        'detection_class_entities': detection_class_entities,
        'detection_class_names': detection_class_names,
        'detection_class_labels': detection_class_labels,
        'detection_scores': detection_scores
    }

# Loop through image files and perform object detection
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    detections = perform_object_detection(image_path)

    print(f"Object detection results for {image_file}:")
    for i in range(5):
        print(f"Detection {i + 1}:")
        print(f"  Bounding Box: {detections['detection_boxes'][i].numpy()}")
        print(f"  Class Entity: {detections['detection_class_entities'][i].numpy()}")
        print(f"  Class Name: {detections['detection_class_names'][i].numpy()}")
        print(f"  Class Label: {detections['detection_class_labels'][i].numpy()}")
        print(f"  Score: {detections['detection_scores'][i].numpy()}")
    print()




In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image
import os
import csv

# Load the object detection model using hub.load with the 'object_detection' signature
detector = hub.load('https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1').signatures['default']

# Path to the directory containing the images
image_dir = '/content/images'

# List all image files in the directory
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Define the expected input size
input_size = (512, 512, 3)

# Load the class descriptions from the CSV file
label_to_category = {}
with open('/content/class-descriptions-boxable.csv', mode='r') as file:
    reader = csv.reader(file)
    detection_class_names = next(reader)  # Read the first row
    label_name = next(reader)  # Read the second row

    # Create a mapping from detection class names to label names
    label_to_category = dict(zip(detection_class_names, label_name))

# Loop through image files and perform object detection for each image
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)

    # Load and preprocess the image
    image = Image.open(image_path)
    # Ensure the image is 512x512
    image = image.resize(input_size[:2])

    # Normalize the pixel values to the range [0.0, 1.0]
    image = np.array(image) / 255.0
    image = image.astype(np.float32)

    # Expand dimensions to match the model's input shape
    image = tf.expand_dims(image, axis=0)

    # Perform object detection using the model
    detector_output = detector(image)

    print(f"Object detection results for {image_file}:")

    # Access all detection information (boxes, class names, and scores)
    detection_boxes = detector_output['detection_boxes'][0].numpy()
    detection_class_names = detector_output['detection_class_names'].numpy()
    detection_scores = detector_output['detection_scores'][0].numpy()

    # Get the number of valid detections
    num_detections = detection_boxes.shape[0]

    # Loop through all detections and print them
    for i in range(num_detections):
        print(f"Detection {i + 1}:")
        print(f"  Bounding Box: {detection_boxes[i]}")

        # Get the class name for this detection
        class_name = detection_class_names[i].decode('utf-8')

        print(f"  Class Name: {class_name}")

        # Get the label name from the label mapping
        label_name = label_to_category.get(class_name)

        print(f"  Label Name: {label_name}")
    print()


Object detection results for frame_430.0.jpg:
Detection 1:
  Bounding Box: 0.23088371753692627
  Class Name: /m/03jm5
  Label Name: None
Detection 2:
  Bounding Box: 0.004010213539004326
  Class Name: /m/0d4v4
  Label Name: None
Detection 3:
  Bounding Box: 0.8722000122070312
  Class Name: /m/01g317
  Label Name: None
Detection 4:
  Bounding Box: 1.0
  Class Name: /m/04bcr3
  Label Name: None

Object detection results for frame_490.0.jpg:
Detection 1:
  Bounding Box: 0.3118939697742462
  Class Name: /m/03jm5
  Label Name: None
Detection 2:
  Bounding Box: 0.5220983624458313
  Class Name: /m/03rszm
  Label Name: None
Detection 3:
  Bounding Box: 0.5753755569458008
  Class Name: /m/07j7r
  Label Name: None
Detection 4:
  Bounding Box: 0.9447689056396484
  Class Name: /m/07j7r
  Label Name: None

Object detection results for frame_90.0.jpg:
Detection 1:
  Bounding Box: 0.33074653148651123
  Class Name: /m/0dzct
  Label Name: None
Detection 2:
  Bounding Box: 0.4508398473262787
  Class Nam

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image
import os
import csv

# Load the object detection model using hub.load with the 'object_detection' signature
detector = hub.load('https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1').signatures['default']

# Path to the directory containing the images
image_dir = '/content/images'

# List all image files in the directory
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Define the expected input size
input_size = (512, 512, 3)

# Load the class descriptions from the CSV file
label_to_category = {}
with open('/content/class-descriptions-boxable.csv', mode='r') as file:
    reader = csv.reader(file)
    for row in reader:
        if len(row) == 2:
            detection_class_names, label_name = row[0], row[1]
            label_to_category[detection_class_names] = label_name

# Loop through image files and perform object detection for each image
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)

    # Load and preprocess the image
    image = Image.open(image_path)
    # Ensure the image is 512x512
    image = image.resize(input_size[:2])

    # Normalize the pixel values to the range [0.0, 1.0]
    image = np.array(image) / 255.0
    image = image.astype(np.float32)

    # Expand dimensions to match the model's input shape
    image = tf.expand_dims(image, axis=0)

    # Perform object detection using the model
    detector_output = detector(image)

    print(f"Object detection results for {image_file}:")

    # Access all detection information (boxes, class names, and scores)
    detection_boxes = detector_output['detection_boxes'][0].numpy()
    detection_class_names = detector_output['detection_class_names'].numpy()
    detection_scores = detector_output['detection_scores'][0].numpy()

    # Get the number of valid detections
    num_detections = detection_boxes.shape[0]

    # Loop through all detections and print them
    for i in range(num_detections):
        print(f"Detection {i + 1}:")
        print(f"  Bounding Box: {detection_boxes[i]}")

        class_label = detection_class_names[i]

        # Get the label name from the label mapping
        label_name = label_to_category.get(class_label, "Unknown")

        print(f"  Class Name: {label_name}")
    print()